In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
data = pd.read_csv('./events/events.csv')
data.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [3]:
data.fillna(int(0),inplace = True)
data.head(10)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,0.0
1,1433224214164,992329,view,248676,0.0
2,1433221999827,111016,view,318965,0.0
3,1433221955914,483717,view,253185,0.0
4,1433221337106,951259,view,367447,0.0
5,1433224086234,972639,view,22556,0.0
6,1433221923240,810725,view,443030,0.0
7,1433223291897,794181,view,439202,0.0
8,1433220899221,824915,view,428805,0.0
9,1433221204592,339335,view,82389,0.0


In [4]:
data.event
data.tail(10)

,timestamp,visitorid,event,itemid,transactionid
2756091,1438398087349,126458,view,439758,0.0
2756092,1438398473572,709520,view,104512,0.0
2756093,1438400498250,226214,view,190530,0.0
2756094,1438399289446,701750,view,296172,0.0
2756095,1438400574346,289041,view,156947,0.0
2756096,1438398785939,591435,view,261427,0.0
2756097,1438399813142,762376,view,115946,0.0
2756098,1438397820527,1251746,view,78144,0.0
2756099,1438398530703,1184451,view,283392,0.0
2756100,1438400163914,199536,view,152913,0.0


In [5]:
data.event = data.event.map({'view':0, 'transaction':1, 'addtocart':2})
print(data.event[0])

0


In [6]:
events_c = data.event.value_counts()
events_c.head()
items_c = data.itemid.value_counts()
items_c.head()

187946    3412
461686    2978
5411      2334
370653    1854
219512    1800
Name: itemid, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder


In [8]:
data.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,0,355908,0.0
1,1433224214164,992329,0,248676,0.0
2,1433221999827,111016,0,318965,0.0
3,1433221955914,483717,0,253185,0.0
4,1433221337106,951259,0,367447,0.0


In [17]:
data_val = data.values
print(data_val.shape)
X = data.loc[:, ['timestamp', 'visitorid', 'itemid','transactionid']]
print(X.shape)
Y = data.loc[:,['event']]
print(Y.shape)

(2756101, 5)
(2756101, 4)
(2756101, 1)


In [39]:
X_val = X.values
Y_val = Y.values
print(X_val.shape, Y_val.shape)

(2756101, 4) (2756101, 1)


In [29]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Sequential

In [30]:
model = Sequential()

In [31]:
embed_dim = 128
lstm_out = 200
batch_size = 32

In [33]:
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """Entry point for launching an IPython kernel.
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  


In [35]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 128)            320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
_________________________________________________________________
embedding_3 (Embedding)      (None, 4, 128)            320000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 1,167,204
Trainable params: 1,167,204
Non-trainable params: 0
_________________________________________________________________


In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_val,Y_val, test_size = 0.20, random_state = 36)

In [45]:
model.fit(X_train, Y_train, batch_size =batch_size, epochs = 5,  verbose = 5)

ValueError: Error when checking target: expected dense_2 to have shape (2,) but got array with shape (1,)